# Lecture 4: Banknote Case

# Banknote Case Study : Decision tree with _k_-fold cross-validation approach
Source: _Machine Learning From Scratch_ (Machine Learning Mastery)

### Imports to read file

In [ ]:
# imports
from random import seed
from random import randrange
from csv import reader

### Function to load the CSV file and create a list of dataset

In [ ]:
def load_csv(filename):

  # empty list of dataset
  dataset = list()    

  with open(filename, 'r') as file:
    csv_reader = reader(file)
    for row in csv_reader:
      if not row:
       continue
      dataset.append(row)

  return dataset

### Function to covert string to float type

In [ ]:
def str_column_to_float(raw_dataset, column):
  for row in raw_dataset:
    row[column] = float(row[column].strip())

# Splitting a dataset & evaluating all splits

### Function to split a dataset 
##### Splits a dataset into two lists of rows given the index of an feature and split value for that particular feature

In [ ]:
def test_split(index, value, dataset):

  left, right = list(), list()
  
  for row in dataset:
    if row[index] < value:
      left.append(row)
    else:
      right.append(row)
  
  return left, right

### Function to calculate Gini Index 

In [ ]:
def gini_index(groups, class_values):

  gini = 0.0

  for class_value in class_values:
    for group in groups:
      size = len(group)
      if size == 0:
        continue
      proportion = [row[-1] for row in group].count(class_value) / float(size)
      gini += (proportion * (1.0 - proportion))

  return gini

### Function to evaluate all splits
##### With the gini_index() and test_split(), get_split() searches best split point of an chosen feature and value of that feature

In [ ]:
# select the best split point for a dataset

def get_split(dataset):

  class_values = list(set(row[-1] for row in dataset))
  b_index, b_value, b_score, b_groups = 999, 999, 999, None

  for index in range(len(dataset[0])-1):
    for row in dataset:
      groups = test_split(index, row[index], dataset)
      gini = gini_index(groups, class_values)
      if gini < b_score:
        b_index, b_value, b_score, b_groups = index, row[index], gini, groups
        
  return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Build a tree: terminal nodes, recurisive splitting, & building a tree


### to_terminal() determines when to stop growing at a given point and is used to make a final prediction 

In [ ]:
# create a terminal node value
def to_terminal(group):
  outcomes = [row[-1] for row in group]
  return max(set(outcomes), key=outcomes.count)

### split() recursively splits a node to create child nodes 

In [ ]:
# create child splits for a node or make terminal

def split(node, max_depth, min_size, depth):

  left, right = node['groups']

  del(node['groups'])

   # check for a no split
  if not left or not right:
    node['left'] = node['right'] = to_terminal(left + right)
    return

  # check for max depth
  if depth >= max_depth:
    node['left'], node['right'] = to_terminal(left), to_terminal(right)
    return

  # process left child
  if len(left) <= min_size:
    node['left'] = to_terminal(left)
  else:
    node['left'] = get_split(left)
    split(node['left'], max_depth, min_size, depth+1) 

  # process right child
  if len(right) <= min_size:
    node['right'] = to_terminal(right)
  else:
    node['right'] = get_split(right)
    split(node['right'], max_depth, min_size, depth+1)


### build_tree() creates a node and calls the split()  that then calls itself recursively to build a whole tree

In [ ]:
# build a decision tree
def build_tree(train, max_depth, min_size):

  root = get_split(train)
  split(root, max_depth, min_size, 1)
  return root
  

### Making a prediction with the specifically provided row of data 

In [ ]:
# make a prediction

def predict(node, row):

  if row[node['index']] < node['value']:
    if isinstance(node['left'], dict):
      return predict(node['left'], row)
    else:
      return node['left']

  else:
    if isinstance(node['right'], dict):
      return predict(node['right'], row)
    else:
      return node['right']
      

# Functions to Implement k-fold cross-validation approach

### Split dataset into _k_-folds defined by user input value


In [ ]:
def cross_validation_split(dataset, n_folds):

  dataset_split = list()
  dataset_copy = list(dataset)
  fold_size = int(len(dataset) / n_folds)

  for i in range(n_folds):
    fold = list()
    while len(fold) < fold_size:
      index = randrange(len(dataset_copy))
      fold.append(dataset_copy.pop(index))
    dataset_split.append(fold)
    
  return dataset_split

### decision_tree() to manage the application of the training and prediction functions


In [ ]:
def decision_tree(train, test, max_depth, min_size):

  tree = build_tree(train, max_depth, min_size)
  predictions = list()

  for row in test:
    prediction = predict(tree, row)
    predictions.append(prediction)
    
  return predictions

### evaluate_algorithm() to evaluate the algorithm with cross-validation  


In [ ]:
# evaluate a tree using cross validation

def evaluate_algorithm(dataset, algorithm, n_folds, *args):

  folds = cross_validation_split(dataset, n_folds)
  scores = list()

  for fold in folds:
    train_set = list(folds)
    train_set.remove(fold)
    train_set = sum(train_set, [])
    test_set = list()
    for row in fold:
      row_copy = list(row)
      test_set.append(row_copy)
      row_copy[-1] = None
    predicted = algorithm(train_set, test_set, *args)
    actual = [row[-1] for row in fold]
    accuracy = accuracy_metric(actual, predicted)
    scores.append(accuracy)
    
  return scores

### accuracy_metrics() to calculate accuracy of prediction

In [ ]:
def accuracy_metric(actual, predicted):

  correct = 0

  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1
      
  return correct / float(len(actual)) * 100.0

# Load banknote data

In [ ]:
# imports to upload the datafile
import io
from google.colab import files  

In [ ]:
# to upload local files
uploaded = files.upload()

Saving data_banknote_authentication.csv to data_banknote_authentication.csv


In [ ]:
# load and prepare data
filename = 'data_banknote_authentication.csv'
raw_dataset = load_csv(filename)
raw_dataset

In [ ]:
# convert string feature values to integers
seed(12345)
for i in range(len(raw_dataset[0])):
  str_column_to_float(raw_dataset, i)
dataset = raw_dataset

## Train decision tree on banknote data

In [ ]:
# define user input values
# build decision tree and evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [81.38686131386861, 85.03649635036497, 81.75182481751825, 83.94160583941606, 83.57664233576642]
Mean Accuracy: 83.139%
